In [2]:
import os
os.chdir("..")  # 回到项目根目录

In [3]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import chardet

# 自动识别编码的读取函数
def read_csv_auto_encoding(filepath):
    with open(filepath, 'rb') as f:
        result = chardet.detect(f.read())
    return pd.read_csv(filepath, encoding=result['encoding'], index_col=0)

# 1. 读取全部文件
area_df = pd.read_excel('outputs/组团土地类型面积统计_统一英文名.xlsx', index_col=0)
planning_w = read_csv_auto_encoding('outputs/planning_weights.csv').squeeze("columns")  # Series
region_w = read_csv_auto_encoding('outputs/region_weights.csv')
scenario_w = read_csv_auto_encoding('outputs/scenario_weights.csv')
total_area = pd.read_excel('outputs/组团区域总面积统计.xlsx', index_col=0)

# 2. 输出结构确认
files = {
    "Area DF": area_df,
    "Planning Weight": planning_w,
    "Region Weight": region_w,
    "Scenario Weight": scenario_w,
    "Total Area": total_area
}

for name, df in files.items():
    print(f"\n====== {name} ======")
    print("Index:", df.index.tolist())
    print("Columns:", df.columns.tolist() if hasattr(df, 'columns') else "This is a Series")



====== Area DF ======
Index: ['Airport New City', 'Fengdong New City', 'Fengxi New City', 'Jinghe New City', 'Qinhan New City', 'Xixian New Area']
Columns: ['Transportation station land', 'Residential land', 'Sports land', 'Park green land', 'Public facility land', 'Agricultural land', 'Agricultural and forestry land', 'Medical land', 'Historical site protection area', 'Commercial land', 'Basic land', 'Basic green land', 'Industrial land', 'Government land', 'Educational land', 'Cultural land', 'Water area', 'Social welfare land']

====== Planning Weight ======
Index: ['Basic land', 'Basic green land', 'Park green land', 'Public facility land', 'Transportation station land', 'Educational land', 'Historical site protection area', 'Agricultural and forestry land', 'Industrial land', 'Agricultural land', 'Commercial land', 'Social welfare land', 'Water area', 'Sports land', 'Cultural land', 'Medical land', 'Government land', 'Residential land']
Columns: This is a Series

====== Region We

In [5]:
import pandas as pd
import numpy as np

# === 1. 读取五个文件 ===
area_df = pd.read_excel('outputs/组团土地类型面积统计_统一英文名.xlsx', index_col=0)
planning_w = pd.read_csv('outputs/planning_weights.csv', index_col=0).squeeze("columns")
region_w = pd.read_csv('outputs/region_weights.csv', index_col=0)
total_area = pd.read_excel('outputs/组团区域总面积统计.xlsx', index_col=0)

# === 2. 统一区域索引（去除 total_area 中的 NaN）===
total_area = total_area[total_area.index.notna()]
area_df = area_df.loc[area_df.index.intersection(total_area.index)]


# === 3. 计算 Aij 比例（面积占比）===
region_total_area = total_area["total_area_km2"]
Aij_ratio = area_df.div(region_total_area, axis=0)

# === 4. 初始化结果表 Sij_df ===
Sij_df = pd.DataFrame(index=area_df.index, columns=planning_w.index)

# === 5. 计算 Sij ===
for land in Sij_df.columns:
    if land not in area_df.columns:
        print(f"[跳过] 地类缺失: {land}")
        continue

    for region in Sij_df.index:
        Aij = Aij_ratio.at[region, land] if land in Aij_ratio.columns else 0
        Wj = planning_w.get(land, 0)
        Cij = region_w.at[land, region] if land in region_w.index and region in region_w.columns else 0

        if Aij <= 0:
            Sij_df.at[region, land] = 0
        else:
            Sij_df.loc[region, land] = Aij * 1000000  * Wj * Cij

Sij_df = Sij_df.astype(float)

# =====【修改处】=====
# 西咸新区（总体）的指标层得分：只采用下面 5 个新城行的平均值
Sij_df.loc["Xixian New Area"] = Sij_df.loc[
    ["Airport New City", "Fengdong New City", "Fengxi New City", "Jinghe New City", "Qinhan New City"]
].mean()


# === 6. 可选：保存或显示结果 ===
display(Sij_df.head())
Sij_df.to_excel('outputs/03_指标层规划得分.xlsx')

,Basic land,Basic green land,Park green land,Public facility land,Transportation station land,Educational land,Historical site protection area,Agricultural and forestry land,Industrial land,Agricultural land,Commercial land,Social welfare land,Water area,Sports land,Cultural land,Medical land,Government land,Residential land
group,,,,,,,,,,,,,,,,,,
Airport New City,0.000000,1.264223,0.841115,0.000000,8.558039,3.321151,0.000000,3.25794,12.699004,96.347307,7.112923,0.078908,0.000000,0.000000,0.075511,1.814714,0.009969,0.877203
Fengdong New City,0.701684,8.400223,16.076929,2.028208,1.113556,9.237747,14.796244,0.00000,0.000000,42.010957,18.520509,1.442148,40.705463,3.526827,4.017144,3.318595,0.212829,12.151404
Fengxi New City,0.000000,5.711218,14.225717,5.653689,0.000000,12.433795,0.000000,0.00000,9.917152,191.664544,7.223620,0.231219,23.118479,0.000000,1.345340,0.817556,0.571310,7.247537
Jinghe New City,0.000000,0.049547,3.608561,1.775576,19.424486,2.860631,0.000000,0.00000,8.888885,216.702558,8.790898,0.000000,6.992493,0.000000,3.478554,1.964905,0.000000,1.140618
Qinhan New City,0.446631,1.102727,1.329685,0.000000,141.670367,2.473068,0.000000,0.00000,8.621955,229.413379,7.090050,0.000000,10.047372,0.000000,0.743816,0.077754,0.197005,1.631081


In [6]:
import pandas as pd
import numpy as np

#------------------------------------------------------------
# 1. 读取仅包含5条区域行(5组团)的Sij文件
#    例如 "Sij_规划指标层得分.xlsx" (放大系数已算入)
#------------------------------------------------------------
Sij_df = pd.read_excel("outputs/03_指标层规划得分.xlsx", index_col=0)

# 显示读取到的行、列情况
print("=== Sij 指标层得分 初始 ===")
print(f"Index: {list(Sij_df.index)}")
print(f"Columns: {list(Sij_df.columns)}\n")



# ================== 4. 目标层得分计算 ==================
# 根据地类分组，定义三大目标层类别（英文名称）：
economic_lands = [
    'Commercial land', 
    'Industrial land'
]
social_lands = [
    'Educational land', 'Government land', 'Medical land', 'Cultural land',
    'Public facility land', 'Social welfare land', 'Residential land',
    'Sports land', 'Transportation station land', 'Historical site protection area'
]
ecological_lands = [
    'Agricultural land', 'Agricultural and forestry land', 'Basic green land',
    'Park green land', 'Water area', 'Basic land'
]

# 目标层得分：对每个区域，分别计算经济、社会、生态得分为对应地类 Sij 的均值
T_df = pd.DataFrame(index=Sij_df.index)
T_df['Economic']   = Sij_df[economic_lands].mean(axis=1)
T_df['Social']     = Sij_df[social_lands].mean(axis=1)
T_df['Ecological'] = Sij_df[ecological_lands].mean(axis=1)
# 区域总得分为三大目标层得分之和
T_df['T_Total']    = T_df['Economic'] + T_df['Social'] + T_df['Ecological']

print("=== 目标层得分预览 ===")
print(T_df.head(6))
print()

#------------------------------------------------------------
# 5. 耦合模型 (只针对「Xixian New Area」)
#   C = (n * (T1*T2*T3)^(1/n)) / (T1+T2+T3)
#   T_norm = (T1+T2+T3) / K   (K=400)
#   D = sqrt(C * T_norm)
#
#   这里T1..T3直接用 Xixian行 [Economic, Social, Ecological]
#   结果写入表的C/T_norm/D列
#------------------------------------------------------------
n = 3
K = 80
xixian_Tk = T_df.loc["Xixian New Area", ['Economic','Social','Ecological']]
C = (n * (xixian_Tk.prod())**(1/n)) / xixian_Tk.sum()
T_norm = xixian_Tk.sum() / K
D = np.sqrt(C * T_norm)

T_df.loc["Xixian New Area", "C"]      = C
T_df.loc["Xixian New Area", "T_norm"] = T_norm
T_df.loc["Xixian New Area", "D"]      = D

print("=== Xixian总区域 耦合模型计算结果 ===")
print("T(k):", xixian_Tk.to_dict())
print(f"C={C:.6f}, T_norm={T_norm:.6f}, D={D:.6f}")

#------------------------------------------------------------
# 6. 保存表格
#------------------------------------------------------------
out_path = "outputs/03_目标层得分_区域总得分_耦合模型.xlsx"
T_df.to_excel(out_path)
print(f"\n已输出: {out_path}")

=== Sij 指标层得分 初始 ===
Index: ['Airport New City', 'Fengdong New City', 'Fengxi New City', 'Jinghe New City', 'Qinhan New City', 'Xixian New Area']
Columns: ['Basic land', 'Basic green land', 'Park green land', 'Public facility land', 'Transportation station land', 'Educational land', 'Historical site protection area', 'Agricultural and forestry land', 'Industrial land', 'Agricultural land', 'Commercial land', 'Social welfare land', 'Water area', 'Sports land', 'Cultural land', 'Medical land', 'Government land', 'Residential land']

=== 目标层得分预览 ===
                   Economic     Social  Ecological    T_Total
group                                                        
Airport New City   9.905964   1.473550   16.951764  28.331277
Fengdong New City  9.260254   5.184470   17.982543  32.427267
Fengxi New City    8.570386   2.830045   39.119993  50.520423
Jinghe New City    8.839891   3.064477   37.892193  49.796562
Qinhan New City    7.856002  14.679309   40.389966  62.925277
Xixian New Ar